In [2]:
from sklearn.ensemble import RandomForestClassifier

import torchvision.models as models
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch
import os
from torchvision import transforms
import pandas as pd
from sklearn.preprocessing import OneHotEncoder


##### Loading the CNN model

In [5]:
BASE_PATH = "./"
TRAINED_DATA_DIR = os.path.join(BASE_PATH, 'DATASETS/merged_resized_pngs_splited_augmented/train')
TEST_DATA_DIR = os.path.join(BASE_PATH, 'DATASETS/merged_resized_pngs_splited_augmented/test')
NUM_CLASSES = len(os.listdir(os.path.join(BASE_PATH, f"{TRAINED_DATA_DIR}")))

print("number of classes: ", NUM_CLASSES)
print("trained data dir: ", TRAINED_DATA_DIR)


def load_model_from_local(model, model_path):
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model


transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MODEL_PATH = 'models/MobileNetV2_20240531_021014/best_model.pth'
mobilenetv2_base_model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)
mobilenetv2_base_model.classifier[1] = nn.Linear(mobilenetv2_base_model.last_channel, NUM_CLASSES)
mobilenetv2_model = load_model_from_local(mobilenetv2_base_model, MODEL_PATH)
mobilenetv2_model.eval()
mobilenetv2_model.to(device)

classes = os.listdir(os.path.join(BASE_PATH, TRAINED_DATA_DIR))
classes.sort()
print("number of classes: ", len(classes))

number of classes:  64
trained data dir:  ./DATASETS/merged_resized_pngs_splited_augmented/train
number of classes:  64


##### Getting the outputs and labels of images with the CNN model
These outputs will be used as features for the classifier

In [48]:
train_dataset = ImageFolder(TRAINED_DATA_DIR, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = ImageFolder(TEST_DATA_DIR, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)


def get_features(model, loader):
    model.eval()  # Ensure the model is in evaluation mode
    features = []
    labels = []
    with torch.no_grad():  # Disable gradient computation
        for images, targets in loader:
            images = images.to(device)
            outputs = model(images)
            features.extend(outputs.cpu().detach().numpy())  # Adjust based on your needs
            labels.extend(targets.cpu().numpy())
    return features, labels

# Example usage

mobilenetv2_features_train, mobilenetv2_labels_train = get_features(mobilenetv2_model, train_loader)
mobilenetv2_features_test, mobilenetv2_labels_test = get_features(mobilenetv2_model, test_loader)


##### One Hot Encoding the plant types and adding it to the features that came from CNN model.

In [58]:
def extract_plant_type(class_name):
    return class_name.split("__")[0]

plant_types = [extract_plant_type(class_name) for class_name in classes]
unique_plant_types = list(set(plant_types))
unique_plant_types.sort() 
print("length of unique plant types: ", len(unique_plant_types))
print("unique plant types: ", unique_plant_types)

length of unique plant types:  15
unique plant types:  ['Apple', 'Cherry', 'Corn', 'Cucumber', 'Grape', 'Peach', 'Pepper_bell', 'Potato', 'Rice', 'Soybean', 'Strawberry', 'Sugarcane', 'Tea', 'Tomato', 'Wheat']


In [57]:
train_df = pd.DataFrame(mobilenetv2_features_train)
train_df['label'] = mobilenetv2_labels_train
train_df['label_name'] = [plant_types[label] for label in mobilenetv2_labels_train]

onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded_train = onehot_encoder.fit_transform(train_df[['label_name']])
onehot_encoded_train_df = pd.DataFrame(onehot_encoded_train, columns=unique_plant_types)
train_df = pd.concat([train_df, onehot_encoded_train_df], axis=1)

train_df = train_df[[col for col in train_df.columns if col != 'label'] + ['label']] # move the label column to the end
train_df = train_df.drop(columns=['label_name']) # remove label name column
train_df.columns = train_df.columns.astype(str)
train_df

c:\Users\90533\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9,...,Pepper_bell,Potato,Rice,Soybean,Strawberry,Sugarcane,Tea,Tomato,Wheat,label
0,-0.165056,-0.538041,0.007287,0.337011,-0.866519,-1.054450,15.817012,1.827885,1.703427,-0.441166,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
1,-1.156583,1.135517,-0.312507,-1.440346,2.477942,-1.931042,-1.120836,-1.431799,0.544684,0.836989,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,33
2,2.756925,2.740755,-1.652043,2.164185,4.547722,-1.964040,-1.726156,-0.797848,0.981734,-1.496274,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,38
3,1.406217,-1.435555,2.362342,3.075113,-2.048617,-0.160708,2.020439,0.983114,-0.865576,-3.607437,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21
4,0.478168,0.599985,2.302902,0.982534,-0.791219,-0.499898,0.322642,0.699635,-0.948669,1.038535,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69821,16.106726,0.897234,3.246588,4.386312,2.470368,3.447539,0.283534,-0.666288,-1.002480,-0.380855,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
69822,-1.324691,1.041057,-1.467048,-1.581200,1.615810,-0.493669,-0.143251,-0.944858,1.866615,-1.214928,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,52
69823,-1.563930,0.105295,-1.109015,-1.628410,0.016121,0.154978,0.652712,6.441171,14.345950,1.025787,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8
69824,-1.388851,0.515232,-0.065401,-0.283708,-0.527374,0.529542,-1.558159,-0.302582,-0.507787,0.687749,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,59


In [62]:
test_df = pd.DataFrame(mobilenetv2_features_test)
test_df['label'] = mobilenetv2_labels_test
test_df['label_name'] = [plant_types[label] for label in mobilenetv2_labels_test]
onehot_encoded_test = onehot_encoder.transform(test_df[['label_name']])
onehot_encoded_test_df = pd.DataFrame(onehot_encoded_test, columns=unique_plant_types)
test_df = pd.concat([test_df, onehot_encoded_test_df], axis=1)

test_df = test_df[[col for col in test_df.columns if col != 'label'] + ['label']] # move the label column to the end
test_df = test_df.drop(columns=['label_name']) # remove label name column
test_df.columns = test_df.columns.astype(str)
test_df



,0,1,2,3,4,5,6,7,8,9,...,Pepper_bell,Potato,Rice,Soybean,Strawberry,Sugarcane,Tea,Tomato,Wheat,label
0,16.737631,-0.392583,3.611225,4.225207,3.443956,2.741155,0.289601,-1.261786,-1.827087,-0.800770,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,18.568239,-0.435375,5.035148,5.359824,3.435232,2.224488,1.799146,-0.473263,-1.712374,-0.486384,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,16.101696,0.072811,3.809797,4.416101,2.900637,3.456900,-0.217153,-1.425478,-1.649548,-0.763041,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,15.639711,0.169368,3.268241,3.275479,3.169479,2.250775,0.386825,-0.789218,-1.098705,-0.723414,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,13.643220,-0.465392,5.949636,3.599722,1.687557,0.846462,3.270836,-0.378807,-1.469336,-2.480021,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17478,-2.324080,-1.436568,1.508470,-2.019795,-2.430835,-1.252507,4.829381,1.653005,0.285291,-0.343349,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,63
17479,-1.731425,-2.005361,0.673403,-1.667158,-1.754823,-1.197402,3.815012,1.711788,0.021841,-0.370908,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,63
17480,-2.118357,-1.818814,3.071315,-2.122261,-2.271806,-1.097417,4.441700,2.078544,-0.105871,-0.332520,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,63
17481,-1.668355,-1.678427,3.196815,-1.300342,-2.298695,-1.243551,5.365029,2.295388,-0.301511,-0.200743,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,63


In [63]:
# save the data
train_df.to_csv("train_df.csv", index=False)
test_df.to_csv("test_df.csv", index=False)
print("data saved")


data saved


##### train a classifier with the features and labels and test the classifier with the test data

In [66]:
rf = RandomForestClassifier(n_estimators=500, max_depth=15, random_state=0)
rf.fit(train_df.drop(columns=['label']), train_df['label'])
print("model trained")

train_accuracy = rf.score(train_df.drop(columns=['label']), train_df['label'])
test_accuracy = rf.score(test_df.drop(columns=['label']), test_df['label'])
print("train accuracy: ", train_accuracy)
print("test accuracy: ", test_accuracy)


model trained
train accuracy:  0.9975367341677885
test accuracy:  0.9636790024595321


In [65]:
# evaluate only cnn model
mobilenetv2_model.eval()
mobilenetv2_model.to(device)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = mobilenetv2_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


print('Accuracy only with cnn: ', correct / total)


Accuracy only with cnn:  0.9587027398043814
